<a href="https://colab.research.google.com/github/sukmoonlee/JupyterNotebook/blob/main/2020/Intel_Xeon_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Intel Xeon CPU 정보 수집용 프로그램
 - https://en.wikipedia.org/wiki/List_of_Intel_Xeon_microprocessors 사이트 정보 이용
 

### 필요 라이브러리 호출

In [ ]:
import pandas as pd
import urllib.request as req
from bs4 import BeautifulSoup

### 메인 페이지에서 요약 정보 추출하기

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_Intel_Xeon_microprocessors"
data = req.urlopen(url).read()
soup = BeautifulSoup(data, "html.parser")
#soup.select("h2 > span.mw-headline")
#soup.select("div > ul > li > a")

In [ ]:
excel = pd.DataFrame()

idx = 0
base_url = dict()

data_tags = soup.select("div > ul > li")
for data_tag in data_tags:  
  try:
    url = data_tag.select_one('a')['href']
    title = data_tag.select_one('a').get('title')
    model = data_tag.select_one('a').string
  except:
    continue

  if (url[0:10]!="/wiki/List") :
    continue
  if (url.find('#')<0) :
    continue
  url = "https://en.wikipedia.org"+url
  attr1 = title.split("of Intel ")
  attr2 = attr1[1].split("-based ")
  base = attr2[0]

  url1 = url.split("#")
  burl = url1[0]
  if burl not in base_url:
    base_url[burl]=burl

  #print(url, "|", title, "|", base, "|", model)
  d=pd.DataFrame()  
  d.loc[idx, 'CPU Model']=model
  d.loc[idx, 'CPU Generation']=base
  d.loc[idx, 'URL']=url
  idx=idx+1
  excel = excel.append(d)

excel.to_excel("CPU_Model_Summary.xls")
excel

,CPU Model,CPU Generation,URL
0,Pentium II Xeon 400,P6,https://en.wikipedia.org/wiki/List_of_Intel_P6...
1,Pentium II Xeon 400,P6,https://en.wikipedia.org/wiki/List_of_Intel_P6...
2,Pentium II Xeon 450,P6,https://en.wikipedia.org/wiki/List_of_Intel_P6...
3,Pentium II Xeon 450,P6,https://en.wikipedia.org/wiki/List_of_Intel_P6...
4,Pentium II Xeon 450,P6,https://en.wikipedia.org/wiki/List_of_Intel_P6...
...,...,...,...
1011,Xeon Platinum 8360HL,Cooper Lake,https://en.wikipedia.org/wiki/List_of_Intel_Co...
1012,Xeon Platinum 8376H,Cooper Lake,https://en.wikipedia.org/wiki/List_of_Intel_Co...
1013,Xeon Platinum 8376HL,Cooper Lake,https://en.wikipedia.org/wiki/List_of_Intel_Co...
1014,Xeon Platinum 8380H,Cooper Lake,https://en.wikipedia.org/wiki/List_of_Intel_Co...


### 상세 페이지에서 세부 정보 가져오기

In [40]:
allexcel = pd.DataFrame()

for key in base_url:
  print(key)
  wikidata = req.urlopen(key).read()
  wikisoup = BeautifulSoup(wikidata, "html.parser")

  table = wikisoup.select("table.wikitable")
  for i in range(0,len(table)):
    df = pd.read_html(str(table))[i]

    if "Modelnumber" in df:
      df.rename(columns={'Modelnumber':'Model Number'}, inplace=True)
    if "sSpec Number" in df:
      df.rename(columns={'sSpec Number':'sSpecnumber'}, inplace=True)
    if "L2cache" in df:
      df.rename(columns={'L2cache':'L2-Cache'}, inplace=True)
    if "L2 Cache" in df:
      df.rename(columns={'L2 Cache':'L2-Cache'}, inplace=True)
    if "L3cache" in df:
      df.rename(columns={'L3cache':'L3-Cache'}, inplace=True)
    if "L3 Cache" in df:
      df.rename(columns={'L3 Cache':'L3-Cache'}, inplace=True)
    if "FSB" in df:
      df.rename(columns={'FSB':'Front Side Bus'}, inplace=True)
    if "Mult." in df:
      df.rename(columns={'Mult.':'Multiplier'}, inplace=True)
    if "Mult" in df:
      df.rename(columns={'Mult':'Multiplier'}, inplace=True)
    if "Release date" in df:
      df.rename(columns={'Release date':'Release Date'}, inplace=True)
    if "Partnumber(s)" in df:
      df.rename(columns={'Partnumber(s)':'Part Number(s)'}, inplace=True)
    if "Releaseprice (USD)" in df:
      df.rename(columns={'Releaseprice (USD)':'Release Price (USD)'}, inplace=True)

    if True in df.columns.str.contains("Unnamed"):
      for n in df.columns:
        if n.find("Unnamed")>=0:          
          del df[n]
   
    allexcel = pd.concat([allexcel, df], ignore_index=True, join='outer')    

allexcel = allexcel.drop( allexcel[allexcel['Model Number']==allexcel['sSpecnumber']].index )
allexcel.to_excel("CPU_Model_All.xls")
allexcel

https://en.wikipedia.org/wiki/List_of_Intel_P6-based_Xeon_microprocessors
https://en.wikipedia.org/wiki/List_of_Intel_NetBurst-based_Xeon_microprocessors
https://en.wikipedia.org/wiki/List_of_Intel_Pentium_M_(Yonah)-based_Xeon_microprocessors
https://en.wikipedia.org/wiki/List_of_Intel_Core-based_Xeon_microprocessors
https://en.wikipedia.org/wiki/List_of_Intel_Nehalem-based_Xeon_microprocessors
https://en.wikipedia.org/wiki/List_of_Intel_Sandy_Bridge-based_Xeon_microprocessors
https://en.wikipedia.org/wiki/List_of_Intel_Ivy_Bridge-based_Xeon_microprocessors
https://en.wikipedia.org/wiki/List_of_Intel_Haswell-based_Xeon_microprocessors
https://en.wikipedia.org/wiki/List_of_Intel_Broadwell-based_Xeon_microprocessors
https://en.wikipedia.org/wiki/List_of_Intel_Skylake-based_Xeon_microprocessors
https://en.wikipedia.org/wiki/List_of_Intel_Kaby_Lake-based_Xeon_microprocessors
https://en.wikipedia.org/wiki/List_of_Intel_Coffee_Lake-based_Xeon_microprocessors
https://en.wikipedia.org/wiki/Lis

,Model Number,sSpecnumber,Frequency,L2-Cache,Front Side Bus,Multiplier,Voltage,TDP,Socket,Release Date,Part Number(s),Release Price (USD),L3-Cache,Cores,Turbo,I/O bus,Memory,GPUfrequency,GPUmodel,Cores(threads),Turbo Boostall-core/2.0(/max. 3.0)
0,Pentium II Xeon 400,SL2RH (B0) SL344 (B0) SL34H (B1) SL35N (B1),400 MHz,512 KiB,100 MT/s,4×,2.0 V,30.8 W,Slot 2,June 1998,80523KX400512 BX80523KX400512 BX80523KX40N512,$1124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pentium II Xeon 400,SL2NB (B0) SL345 (B0) SL34J (B1) SL35P (B1),400 MHz,1 MiB,100 MT/s,4×,2.0 V,38.1 W,Slot 2,June 1998,80523KX4001M BX80523KX40 BX80523KX40,$2836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pentium II Xeon 450,SL2XJ (B1) SL354 (B1) SL36W (B1) SL33T (B1),450 MHz,512 KiB,100 MT/s,4.5×,2.0 V,34.5 W,Slot 2,October 1998,80523KX450512 BX80523KX450512 BX80523KX45G512,$824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pentium II Xeon 450,SL2XK (B1) SL33U (B1),450 MHz,1 MiB,100 MT/s,4.5×,2.0 V,42.8 W,Slot 2,January 1999,80523KX4501M BX80523KX4501M,$1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pentium II Xeon 450,SL2XL (B1) SL33V (B1),450 MHz,2 MiB,100 MT/s,4.5×,2.0 V,46.7 W,Slot 2,January 1999,80523KX4502M BX80523KX4502M,$3692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,Xeon Platinum 8360HL,SRK58 (A1),3 GHz,24 × 1 MiB,NaN,NaN,NaN,225 W,LGA 4189,1 September 2020,CD8070604559801,$7203,33 MiB,NaN,NaN,6× 10.4 GT/s UPI,6× DDR4-3200,NaN,NaN,24 (48),?/4.2 GHz
1243,Xeon Platinum 8376H,SRJXS (A1),2.6 GHz,28 × 1 MiB,NaN,NaN,NaN,205 W,LGA 4189,18 June 2020,CD8070604480501,$8719,38.5 MiB,NaN,NaN,6× 10.4 GT/s UPI,6× DDR4-3200,NaN,NaN,28 (56),?/4.3 GHz
1244,Xeon Platinum 8376HL,SRJXT (A1),2.6 GHz,28 × 1 MiB,NaN,NaN,NaN,205 W,LGA 4189,18 June 2020,CD8070604480601,"$11,722",38.5 MiB,NaN,NaN,6× 10.4 GT/s UPI,6× DDR4-3200,NaN,NaN,28 (56),?/4.3 GHz
1245,Xeon Platinum 8380H,SRJXQ (A1),2.9 GHz,28 × 1 MiB,NaN,NaN,NaN,250 W,LGA 4189,18 June 2020,CD8070604480301,"$10,009",38.5 MiB,NaN,NaN,6× 10.4 GT/s UPI,6× DDR4-3200,NaN,NaN,28 (56),?/4.3 GHz


In [42]:
# allexcel = allexcel.drop(allexcel[allexcel['Model Number'].str.contains("core")].index)
# allexcel = allexcel.drop(allexcel[allexcel['Model Number'].str.contains("Core")].index)
# allexcel = allexcel.drop(allexcel[allexcel['Model Number'].str.contains("standard power")].index)
# allexcel = allexcel.drop(allexcel[allexcel['Model Number'].str.contains("Edge Server and Cloud")].index)
# allexcel = allexcel.drop(allexcel[allexcel['Model Number'].str.contains("Network")].index)

# allexcel = allexcel.drop( allexcel[allexcel['Model Number']==allexcel['sSpecnumber']].index )
# allexcel.to_excel("CPU_Model_All.xls")
# allexcel

if True in allexcel.columns.str.contains("Unnamed"):
  print("Unnamed")